In [4]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [5]:
import pandas as pd

from wikilite.utils import wiktextract

if __name__ == "__main__":
    words = []
    forms = {}
    for page in wiktextract.load():
        forms[page.word] = forms.get(page.word, [page.word]) + [
            f["form"] for f in page.forms
        ]
        word = page.word
        if len(page.senses) == 0:
            words.append((word, word))
        else:
            for sense in page.senses:
                sense_glosses = sense.raw_glosses
                if len(sense_glosses) == 0 and sense.glosses:
                    sense_glosses = sense.glosses
                words.append((word, " ".join(sense_glosses)))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1337403/1337403 [00:54<00:00, 24739.42it/s]


In [96]:
forms_df = pd.DataFrame(forms.items(), columns=["word", "forms"])
forms_df = forms_df.explode("forms")
words_df = pd.DataFrame(words, columns=["word", "definition"])
words_df = words_df.drop_duplicates(["word", "definition"])

In [97]:
words_df["text"] = words_df["word"] + ". " + words_df["definition"]

In [99]:
# .drop(columns="id")
words_df = words_df.reset_index(drop=True).reset_index(names="id")

In [26]:
from retriv import SearchEngine

collection = words_df.to_dict(orient="records")

se = SearchEngine("new-index").index(collection)

Building inverted index: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 541589/541589 [00:30<00:00, 17765.45it/s]


In [28]:
if __name__ == "__main__":
    rel_senses = []
    for word in wiktextract.load():
        all_relations = []
        for s in wiktextract.get_relations(word).values():
            all_relations.extend(s)
        for sense in word.senses:
            for s in wiktextract.get_relations(sense).values():
                all_relations.extend(s)
        for rel in all_relations:
            if rel.sense:
                rel_senses.append((rel.word, rel.sense))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1337403/1337403 [00:54<00:00, 24355.98it/s]


In [115]:
rel_senses_df = pd.DataFrame(rel_senses, columns=["word", "sense"]).drop_duplicates()
rel_senses_df = rel_senses_df.reset_index(drop=True).reset_index(names="id")
rel_senses_df["text"] = (
    rel_senses_df["word"] + ". " + rel_senses_df["sense"]
).str.strip()


In [116]:
# rel_senses_df["matches"] = rel_senses_df["text"].progress_apply(se.search)

In [117]:
# rel_senses_df["matches"].head()

In [118]:
# def get_word_matches(row):
#     results = []
#     word = row["word"].lower()
#     for item in row["matches"]:
#         if item["word"].lower() == word:
#             results.append(item)
#     if len(results) > 0:
#         return results[0]["definition"]
#     return None

# rel_senses_df["matches_filtered"] = rel_senses_df.apply(get_word_matches, axis=1)

In [119]:
# rel_senses_df = rel_senses_df.drop(columns=["matches_filtered"])
# "matches_0", "matches_0_id", 

In [120]:
# pd.merge(rel_senses_df, words_df, how="left", right_on="id", left_on="matches_0_id", suffixes=["", "_"])

In [121]:
# rel_senses_df[rel_senses_df["matches_filtered"].isna()]

In [122]:
rel_senses_df.head()

,id,word,sense,text
0,0,diarthrosis,"an articulation, especially one that permits f...","diarthrosis. an articulation, especially one t..."
1,1,aberrant,abnormality—something that is abnormal,aberrant. abnormality—something that is abnormal
2,2,abnormality,abnormality—state of being abnormal,abnormality. abnormality—state of being abnormal
3,3,spontaneity,suddenness,spontaneity. suddenness
4,4,augmentation,antonym(s) of “lessening”,augmentation. antonym(s) of “lessening”


In [128]:
words_df[["word", "definition"]].duplicated().any()

False

In [124]:
rel_senses_df = pd.merge(rel_senses_df, words_df, how="left", on="word", suffixes=["", "_"])

In [129]:
rel_senses_df[["word", "sense"]].duplicated().any()

True

In [130]:
rel_senses_df.head()

,id,word,sense,text,id_,definition,text_
0,0,diarthrosis,"an articulation, especially one that permits f...","diarthrosis. an articulation, especially one t...",575801.0,(anatomy) A joint that can move freely in vari...,diarthrosis. (anatomy) A joint that can move f...
1,1,aberrant,abnormality—something that is abnormal,aberrant. abnormality—something that is abnormal,1121.0,Differing from the norm.,aberrant. Differing from the norm.
2,1,aberrant,abnormality—something that is abnormal,aberrant. abnormality—something that is abnormal,1122.0,(sometimes figuratively) Straying from the rig...,aberrant. (sometimes figuratively) Straying fr...
3,1,aberrant,abnormality—something that is abnormal,aberrant. abnormality—something that is abnormal,1123.0,"(botany, zoology) Deviating from the ordinary ...","aberrant. (botany, zoology) Deviating from the..."
4,1,aberrant,abnormality—something that is abnormal,aberrant. abnormality—something that is abnormal,1124.0,A person or object that deviates from the rest...,aberrant. A person or object that deviates fro...
